# Predicting NYC Taxi Fares with RAPIDS

[RAPIDS](https://rapids.ai/) is a suite of GPU accelerated data science libraries with APIs that should be familiar to users of Pandas, Dask, and Scikitlearn.

Anaconda has graciously made some of the NYC Taxi dataset available in [a public Google Cloud Storage bucket](https://console.cloud.google.com/storage/browser/anaconda-public-data/nyc-taxi/csv/).

This notebook builds a simple data pipeline to load the data with cuDF (or Pandas) and saves it back a cleaned-up CSV for further analysis.

In [1]:
import numpy as np
import pandas as pd
import cuml
import cudf
import os

# Inspecting the Data

Let's start with a familiar Pandas approach then port it to RAPIDS in parallel

In [2]:
!ls ../../data/nyc-taxi/2014

yellow_tripdata_2014-01.csv  yellow_tripdata_2014-07.csv
yellow_tripdata_2014-02.csv  yellow_tripdata_2014-08.csv
yellow_tripdata_2014-03.csv  yellow_tripdata_2014-09.csv
yellow_tripdata_2014-04.csv  yellow_tripdata_2014-10.csv
yellow_tripdata_2014-05.csv  yellow_tripdata_2014-11.csv
yellow_tripdata_2014-06.csv  yellow_tripdata_2014-12.csv


In [3]:
base_path = '../../data/nyc-taxi/'

In [4]:
%%time

gdf_2014 = cudf.read_csv(base_path+'2014/yellow_tripdata_2014-03.csv')
gdf_2014.head()

CPU times: user 2.04 s, sys: 1.01 s, total: 3.05 s
Wall time: 3.08 s


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-03-01 01:07:38,2014-03-01 01:16:26,1,2.0,-73.951504,40.714113,1,N,-73.944278,40.690677,CRD,9.0,0.5,0.5,2.0,0.0,12.0
1,CMT,2014-03-01 01:08:03,2014-03-01 01:12:51,2,1.2,-74.004735,40.742173,1,N,-73.992228,40.748210,CRD,6.0,0.5,0.5,1.0,0.0,8.0
2,CMT,2014-03-01 01:08:51,2014-03-01 01:13:18,3,0.5,-73.949302,40.707569,1,N,-73.951654,40.714250,CRD,5.0,0.5,0.5,1.2,0.0,7.2
3,CMT,2014-03-01 01:09:20,2014-03-01 01:24:18,3,3.5,-73.993539,40.721300,1,N,-73.961437,40.760412,CRD,14.0,0.5,0.5,3.0,0.0,18.0
4,CMT,2014-03-01 01:09:46,2014-03-01 01:22:34,1,1.8,-73.987881,40.744579,1,N,-74.004384,40.722775,CRD,10.5,0.5,0.5,1.0,0.0,12.5


In [5]:
gdf_2014.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                         'ratecodeid': 'rate_code',
                         'tpep_dropoff_datetime': 'dropoff_datetime'
                        })

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-03-01 01:07:38,2014-03-01 01:16:26,1,2.0,-73.951504,40.714113,1,N,-73.944278,40.690677,CRD,9.0,0.5,0.5,2.0,0.0,12.0
1,CMT,2014-03-01 01:08:03,2014-03-01 01:12:51,2,1.2,-74.004735,40.742173,1,N,-73.992228,40.748210,CRD,6.0,0.5,0.5,1.0,0.0,8.0
2,CMT,2014-03-01 01:08:51,2014-03-01 01:13:18,3,0.5,-73.949302,40.707569,1,N,-73.951654,40.714250,CRD,5.0,0.5,0.5,1.2,0.0,7.2
3,CMT,2014-03-01 01:09:20,2014-03-01 01:24:18,3,3.5,-73.993539,40.721300,1,N,-73.961437,40.760412,CRD,14.0,0.5,0.5,3.0,0.0,18.0
4,CMT,2014-03-01 01:09:46,2014-03-01 01:22:34,1,1.8,-73.987881,40.744579,1,N,-74.004384,40.722775,CRD,10.5,0.5,0.5,1.0,0.0,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15428122,CMT,2014-03-18 18:14:19,2014-03-18 18:27:22,1,0.3,-73.987367,40.739568,1,N,-74.008448,40.743081,CRD,3.5,1.0,0.5,6.0,0.0,11.0
15428123,CMT,2014-03-18 10:12:33,2014-03-18 10:28:09,1,3.3,-73.973571,40.789391,1,N,-73.958701,40.760576,CRD,13.5,0.0,0.5,4.2,0.0,18.2
15428124,CMT,2014-03-18 09:02:37,2014-03-18 09:16:29,1,6.2,-73.977260,40.749512,1,N,-74.013530,40.714990,CRD,19.5,0.0,0.5,1.0,0.0,21.0
15428125,CMT,2014-03-18 10:10:19,2014-03-18 10:19:25,1,1.7,-73.981583,40.769048,1,N,-73.962727,40.773948,CRD,8.5,0.0,0.5,1.8,0.0,10.8


# Data Cleanup

As usual, the data needs to be massaged a bit before we can start adding features that are useful to an ML model.

For example, in the 2014 taxi CSV files, there are `pickup_datetime` and `dropoff_datetime` columns.

Also, some CSV files have column names with extraneous spaces in them.

We'll do a little string manipulation, column renaming, and concatenating of DataFrames to sidestep the problems.

In [6]:
# list of column names that need to be re-mapped
remap = {}
remap['tpep_pickup_datetime'] = 'pickup_datetime'
remap['tpep_dropoff_datetime'] = 'dropoff_datetime'
remap['ratecodeid'] = 'rate_code'

# create a list of columns & dtypes the df must have
# note that float64 will be significantly slower on some GPUs (most GeForce, also Tesla T4)
must_haves = {
 'pickup_datetime': 'datetime64[ms]',
 'dropoff_datetime': 'datetime64[ms]',
 'passenger_count': 'int32',
 'trip_distance': 'float',
 'pickup_longitude': 'float',
 'pickup_latitude': 'float',
 'rate_code': 'int32',
 'dropoff_longitude': 'float',
 'dropoff_latitude': 'float',
 'fare_amount': 'float'
}

In [7]:
# helper function which takes a DataFrame and fixes column types
def clean_columns(df_part, remap, must_haves, float_type):    
    # iterate through columns in this df
    for col in df_part.columns:
        # drop anything not in our expected list
        if col not in must_haves:
            print(f"Dropping ({col})")
            df_part = df_part.drop(columns=col)
            continue
        
        if df_part[col].dtype == 'object' and col in ['pickup_datetime', 'dropoff_datetime']:
            df_part[col] = df_part[col].astype('datetime64[ms]')
            continue
                
        # if column was read as a string, recast as float
        if df_part[col].dtype == 'object':
            df_part[col] = df_part[col].str.fillna('-1')
            df_part[col] = df_part[col].astype('float32')
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if 'int' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('int32')
            if 'float' in str(df_part[col].dtype):
                # CPU-based pandas has a bug preventing query use with fp32 columns
                df_part[col] = df_part[col].astype(float_type)
            df_part[col] = df_part[col].fillna(-1)
    
    return df_part

In [8]:
%%time
# some col-names include pre-pended spaces remove & lowercase column names
col_cleanup = {col: col.strip().lower() for col in list(gdf_2014.columns)}
gdf = gdf_2014.rename(columns=col_cleanup)
# rename columns using the supplied mapping
gdf = gdf.rename(columns=remap)

gdf = clean_columns(gdf, remap, must_haves, np.float32)
print(gdf.__class__)
print(gdf.head(1))

Dropping (vendor_id)
Dropping (store_and_fwd_flag)
Dropping (payment_type)
Dropping (surcharge)
Dropping (mta_tax)
Dropping (tip_amount)
Dropping (tolls_amount)
Dropping (total_amount)
<class 'cudf.core.dataframe.DataFrame'>
      pickup_datetime    dropoff_datetime  passenger_count  trip_distance  \
0 2014-03-01 01:07:38 2014-03-01 01:16:26                1            2.0   

   pickup_longitude  pickup_latitude  rate_code  dropoff_longitude  \
0        -73.951508        40.714111          1         -73.944275   

   dropoff_latitude  fare_amount  
0         40.690678          9.0  
CPU times: user 93.8 ms, sys: 371 ms, total: 465 ms
Wall time: 464 ms


In [9]:
# gdf.to_csv("yellow_tripdata_2014-03-cleaned.csv", index=False)

In [10]:
# gdf.to_pandas().to_parquet("yellow_tripdata_2014-03-cleaned.parquet", index=False)

In [11]:
gdf.to_orc("yellow_tripdata_2014-03-cleaned.orc", index=False)

In [12]:
gdf.dtypes

pickup_datetime      datetime64[ms]
dropoff_datetime     datetime64[ms]
passenger_count               int32
trip_distance               float32
pickup_longitude            float32
pickup_latitude             float32
rate_code                     int32
dropoff_longitude           float32
dropoff_latitude            float32
fare_amount                 float32
dtype: object